In [50]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc

from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Add, Multiply
from keras.layers import ELU, PReLU, LeakyReLU
from keras.models import Model
from keras.utils.vis_utils import plot_model
from tensorflow.keras import optimizers
from keras.optimizers import SGD
from keras.optimizers import Adagrad
from keras import metrics
from sklearn.metrics import r2_score
from keras.callbacks import EarlyStopping
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121

from tensorflow.keras.layers import GlobalAveragePooling2D

from keras import backend as K

from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import math as m
import numpy as np
import seaborn as sns
import os
import json

sns.set_style("darkgrid")
edgecolors=None

config = tf.compat.v1.ConfigProto()

config.gpu_options.allow_growth=True

sess = tf.compat.v1.Session(config=config)


class AutomaticLearning:



  def __init__(self , train_images, train_labels, test_images, test_labels, title, model_name):

    #print(dataframe.head(2))

    #self.df = dataframe
    self.train_images = train_images
    self.train_labels = train_labels
    self.test_images = test_images
    self.test_labels = test_labels

    self.model_name = model_name
    self.input_shape = None
    self.num_classes = None
    self.epochs = None
    self.model = None
    self.X = None
    self.y = None
    self.X_train = None
    self.X_test = None
    self.y_train = None
    self.y_test = None
    self.run_name = "run"
    self.history = None
    self.r2 = None
    self.loss = None
    self.mse = None
    self.mae = None
    self.rmse = None
    self.library = None
    self.title = title
    self.plottittle = ""
    self.save = None
    self.show = None
    self.y_pred = None
    return

  def gather_data(self):
    return {'r2': self.r2, 'loss': self.loss, 'mse': self.mse, 'mae': self.mae, 'rmse': self.rmse}

  def plotA(self):
    pd.DataFrame(self.history.history).plot(figsize=(8,5))
    plt.show()
    return

  def plotLinear(self):
    # plot the data and the predictions

    #print(self.X_test)
    predictions = self.model.predict(self.X_test)

    plt.xlabel('Area (pixels^2)')
    plt.ylabel('Mass (g)')

    plt.scatter(self.X_test, self.y_test)
    if(self.model_name == "SVR"):
      plt.scatter(self.X_test, predictions, color='red', marker="x")
    else:
      plt.plot(self.X_test, predictions, color='red')

    plt.title(self.plottittle +'-R2' + str(self.epochs), size = 20)

    if(self.show):
      plt.show()
      print()
    if(self.save):
      plt.savefig(self.folder +'/'+self.plottittle + '-R2' + str(self.epochs), dpi=200)
    #print(f'R-squared value: {self.r2:.4f}')
    plt.clf()


    plt.scatter(self.y_test, predictions)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.text(0.05, 0.95, 'R^2 = {:.4f}'.format(self.r2), ha='left', va='center', transform=plt.gca().transAxes)
    plt.title(self.plottittle + '-Predictions', size = 20)
    if(self.show):
      print()
      plt.show()
    if(self.save):
      plt.savefig(self.folder +'/'+self.plottittle + '-Predictions'+ str(self.epochs), dpi=200)
    plt.clf()


    return

  def plotLoss(self):
    fig, ax = plt.subplots()
    ax.set(title='Loss')
    ax.plot(self.history.history['loss'], label='Loss')
    ax.plot(self.history.history['val_loss'], label='Val loss')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Values')
    ax.legend()
    plt.show()
    return

  def plotMSE(self):
    fig, ax = plt.subplots()
    ax.set(title='MSE')
    ax.plot(self.history.history['mse'], label='MSE')
    ax.plot(self.history.history['val_mse'], label='Val MSE')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Values')
    ax.legend()
    plt.show()
    return
    

  def plotMAE(self):
    fig, ax = plt.subplots()
    ax.set(title='MAE')
    ax.plot(self.history.history['mae'], label='MAE')
    ax.plot(self.history.history['val_mae'], label='Val MAE')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Values')
    ax.legend()
    plt.show()
    return

  def plotRMSE(self):
    fig, ax = plt.subplots()
    ax.set(title='RMSE')
    ax.plot(self.history.history['root_mean_squared_error'], label='RMSE')
    ax.plot(self.history.history['val_root_mean_squared_error'], label='Val RMSE')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Values')
    ax.legend()
    plt.show()
    return
  
  def printAccuracy(self):

    accuracy = self.model.evaluate(self.test_images, self.test_labels)
    print('Test Accuracy:', accuracy)

  def plotAccuracy(self):
    plt.plot(self.history.history['accuracy'])  # Use 'acc' instead of 'accuracy'
    plt.plot(self.history.history['val_accuracy'])  # Use 'val_acc' instead of 'val_accuracy'
    #plt.title('Training and Validation Accuracy')
    plt.title(self.plottittle + '-Training and Validation Accuracy', size = 20)

    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Validation'])
    if(self.save):
      plt.savefig(self.folder +'/'+self.plottittle + '-Accuracy'+ str(self.epochs), dpi=300)
    plt.clf()

    #plt.show()

  def plotROC(self):
    # Step 1: Make predictions


    # Step 2: Compute prediction probabilities for the positive class
    y_pred_positive = self.y_pred[:, 1]

    # Step 3: Compute ROC curve metrics
    fpr, tpr, thresholds = roc_curve(self.y_train, y_pred_positive)
    roc_auc = auc(fpr, tpr)

    # Step 4: Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # Random guessing line
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    #plt.show()

  def plotClas(self):
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    #predictions = model.predict(x_test)
    self.y_pred = self.model.predict(self.test_images)

    predicted_labels = np.argmax(self.y_pred, axis=1)

    # Plot a grid of images with their predicted labels
    plt.subplots_adjust(hspace=0.5, wspace=0.5)

    plt.figure(figsize=(12, 14))
    plt.title(self.plottittle + '-Classification', size = 20)

    for i in range(25):
        

      plt.subplot(5, 5, i + 1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)



      image = (self.test_images[i] - np.min(self.test_images[i])) / (np.max(self.test_images[i]) - np.min(self.test_images[i]))
      plt.imshow(image, cmap=plt.cm.gray)  # Change the color map if needed
      predicted_label = predicted_labels[i]
      true_label = self.test_labels[i]

 
      if predicted_label == true_label:
          color = 'green'
      else:
          color = 'red'

      plt.xlabel(f"Predicted: {class_names[predicted_label]}\nTrue: {class_names[true_label]}", color=color)
    #plt.show()


    if(self.save):
      plt.savefig(self.folder +'/'+self.plottittle + '-Classification'+ str(self.epochs), dpi=300)


    if(self.show):
      #plt.show()
      print()
    plt.clf()
  
  def plotPredictions(self):
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


    self.y_pred = self.model.predict(self.test_images)
    y_pred_classes = np.argmax(self.y_pred, axis=1)

    # Generate the confusion matrix
    cm = confusion_matrix(self.test_labels, y_pred_classes)


    #print(f'R-squared value: {self.r2:.4f}')

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)

    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')

    plt.title(self.plottittle + '-Confusion Matrix', size = 20)

    if(self.save):
      plt.savefig(self.folder +'/'+self.plottittle + '-Confusion Matrix'+ str(self.epochs), dpi=300)


    if(self.show):
      #plt.show()
      print()
    plt.clf()


  def plotAll(self):

    fig, ax = plt.subplots(nrows = 2, ncols = 2, figsize = (15,15))
   
    fig.suptitle(self.plottittle, size = 35)   

    ax[0][0].set(title='Loss')
    ax[0][0].plot(self.history.history['loss'], label='Loss')
    ax[0][0].plot(self.history.history['val_loss'], label='Val loss')
    ax[0][0].set_xlabel('Epoch')
    ax[0][0].set_ylabel('Values')
    ax[0][0].legend()


    ax[0][1].set(title='MSE')
    ax[0][1].plot(self.history.history['mse'], label='MSE')
    ax[0][1].plot(self.history.history['val_mse'], label='Val MSE')
    ax[0][1].set_xlabel('Epoch')
    ax[0][1].set_ylabel('Values')
    ax[0][1].legend()

   

    ax[1][0].set(title='MAE')
    ax[1][0].plot(self.history.history['mae'], label='MAE')
    ax[1][0].plot(self.history.history['val_mae'], label='Val MAE')
    ax[1][0].set_xlabel('Epoch')
    ax[1][0].set_ylabel('Values')
    ax[1][0].legend()

    ax[1][1].set(title='RMSE')
    ax[1][1].plot(self.history.history['root_mean_squared_error'], label='RMSE')
    ax[1][1].plot(self.history.history['val_root_mean_squared_error'], label='Val RMSE')
    ax[1][1].set_xlabel('Epoch')
    ax[1][1].set_ylabel('Values')
    ax[1][1].legend()

    if(self.save):
      #print("IM SAVING", self.save)

      plt.savefig(self.folder +'/'+self.plottittle+'-Summary'+ str(self.epochs), dpi=200)

    if(self.show):
      #print("IM SHOWING", self.show)

      plt.show()
    plt.clf()


  def plotNonLinear(self):
    print(self.history.history.keys())

    self.plotAll()
    self.plotPredictions()
    self.printAccuracy()
    self.plotAccuracy()
    self.plotClas()
    #self.plotROC()
    #self.plotLoss()
    #self.plotMAE()
    #self.plotMSE()
    #self.plotRMSE()
    return

  def plot(self, folder, save, show):
    self.save = save
    self.show = show
    self.folder = folder
    if not os.path.exists(self.folder):
      #print("creating folder:", self.folder)
      os.makedirs(self.folder)

    if(self.library == "Tensorflow"):
      if(self.save and self.show): 
        print(self.model.summary())
        #plot_model(self.folder + '/' + self.model, to_file= self.model_name + "-" + self.run_name + "-" + self.title + '.png', show_shapes=True, show_layer_names=False)
        #plot_model(self.model, show_shapes=True, show_layer_names=False)
      if(self.save):
        print("IM SAVING")
        #img = mpimg.imread(self.title +'/'+ self.model_name + "-" + self.run_name + "-" + self.title + '.png')
      if(self.show):
        print("IM SHOWING")
        #imgplot = plt.imshow(img)
        #imgplot = plt.imshow(img2)
        #plt.show()


    if(self.library == "Tensorflow"):
      self.plotNonLinear()
    elif(self.library == "scikit"):
      #print("plotting linear")
      self.plotLinear()


  def train(self, epochs = 2):

    self.epochs = epochs
    self.plottittle = self.title

    if(self.library == "Tensorflow"):
      opt = optimizers.Adam(learning_rate=0.01)
      sgd = optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
      
      self.model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer = opt, metrics=['mse', 'mae','accuracy', tf.keras.metrics.RootMeanSquaredError()])      
      early_stop = EarlyStopping(monitor='val_loss', patience=250, verbose=1)

      self.history = self.model.fit(self.X_train, self.y_train, batch_size=32, epochs = self.epochs, validation_split=0.2, shuffle = True,callbacks=[early_stop], verbose = 1)
      K.clear_session()
      #['loss', 'mse', 'mae', 'root_mean_squared_error']
      results = self.model.evaluate(self.X_test, self.y_test)
      self.loss = results[0]
      self.mse = results[1]
      self.mae = results[2]
      self.rmse = results[3]
   
    return
  
  def buildData(self):

    normalizer = Normalization()

    if(self.model_name == 'CNN' or self.model_name == 'CNN2' or self.model_name == 'ResNet' or self.model_name == 'MobileNet' or self.model_name == 'DenseNet'):

      
      self.train_images = np.expand_dims(self.train_images, axis=-1)
      self.train_images = np.repeat(self.train_images, 3, axis=-1)
      self.train_images = keras.applications.mobilenet.preprocess_input(self.train_images)
      self.train_images = tf.image.resize(self.train_images, (32, 32))
      self.test_images = np.expand_dims(self.test_images, axis=-1)
      self.test_images = np.repeat(self.test_images, 3, axis=-1)
      self.test_images = keras.applications.mobilenet.preprocess_input(self.test_images)
      self.test_images = tf.image.resize(self.test_images, (32, 32))


      self.train_images = self.train_images / 255.0
      self.test_images = self.test_images / 255.0
      print()
        

    self.X_train = self.train_images
    self.X_test = self.test_images

    self.y_train = self.train_labels
    self.y_test = self.test_labels

    self.input_shape = (32, 32, 3)
    num_classes = 1
    return


  def buildModel(self):
    if(self.model_name == "CNN2"):
      print("training cnn2")
      self.library = "Tensorflow"

      self.model = keras.Sequential([
          keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)),
          keras.layers.Conv2D(64, (3, 3), activation='relu'),
          keras.layers.BatchNormalization(),
          keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
          keras.layers.Conv2D(128, (3, 3), activation='relu'),
          keras.layers.Conv2D(128, (3, 3), activation='relu'),
          keras.layers.BatchNormalization(),
          keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
          keras.layers.Conv2D(256, (3, 3), activation='relu'),
          keras.layers.Conv2D(256, (3, 3), activation='relu'),
          keras.layers.BatchNormalization(),
          keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same'),
          keras.layers.Flatten(),
          keras.layers.Dense(512, activation='relu'),
          keras.layers.Dropout(0.5),
          keras.layers.Dense(256, activation='relu'),
          keras.layers.Dropout(0.5),
          keras.layers.Dense(10, activation='softmax')
      ])
    elif(self.model_name == "CNN"):
      print("training cnnn")
      self.library = "Tensorflow"
      self.model = keras.Sequential([
          keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
          keras.layers.MaxPooling2D((2, 2)),
          keras.layers.Conv2D(64, (3, 3), activation='relu'),
          keras.layers.MaxPooling2D((2, 2)),
          keras.layers.Flatten(),
          keras.layers.Dense(64, activation='relu'),
          keras.layers.Dense(10, activation='softmax')
      ])
    elif(self.model_name == "ResNet"):
      self.library = "Tensorflow"
      base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

      x = base_model.output
      x = GlobalAveragePooling2D()(x)
      x = Dense(1, activation='linear')(x)
      predictions = Dense(10, activation='softmax')(x)
      self.model = Model(inputs=base_model.input, outputs=predictions)
      for layer in base_model.layers:
          layer.trainable = False


    elif(self.model_name == "DenseNet"):
      self.library = "Tensorflow"

      base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

      x = base_model.output
      x = GlobalAveragePooling2D()(x)
      x = Dense(1, activation='linear')(x)
      predictions = Dense(10, activation='softmax')(x)
      self.model = Model(inputs=base_model.input, outputs=predictions)
      for layer in base_model.layers:
          layer.trainable = False

    elif(self.model_name == "MobileNet"):
      self.library = "Tensorflow"

      base_model = MobileNet(weights='imagenet',include_top=False, input_shape=(32, 32, 3))
      x = base_model.output
      x = GlobalAveragePooling2D()(x)
      x = Dense(1, activation='linear')(x)
      predictions = Dense(10, activation='softmax')(x)
      self.model = Model(inputs=base_model.input, outputs=predictions)
      for layer in base_model.layers:
          layer.trainable = False
    return




2023-06-06 10:47:50.801818: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-06 10:47:50.802006: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Loading the Fashion MNIST dataset

In [24]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Runs using ADAM

In [52]:
model_name = "CNN"
number_of_epochs = 20

AL = AutomaticLearning(train_images, train_labels, test_images, test_labels, model_name + " to " + str(number_of_epochs) + " epochs", model_name)
AL.buildData()
AL.buildModel()
AL.train(number_of_epochs)
AL.plot(model_name, save = True, show = False)
result = AL.gather_data()


training cnnn
Epoch 1/20


/Users/pablo/Library/Python/3.9/lib/python/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-06-06 10:51:42.381034: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1498/1500 [============================>.] - ETA: 0s - loss: 0.6992 - mse: 27.7352 - mae: 4.4293 - accuracy: 0.7341 - root_mean_squared_error: 5.2664

2023-06-06 10:52:01.965169: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1500/1500 [==============================] - 23s 14ms/step - loss: 0.6991 - mse: 27.7358 - mae: 4.4294 - accuracy: 0.7341 - root_mean_squared_error: 5.2665 - val_loss: 0.5280 - val_mse: 27.3835 - val_mae: 4.3824 - val_accuracy: 0.8033 - val_root_mean_squared_error: 5.2329
Epoch 2/20
1500/1500 [==============================] - 20s 13ms/step - loss: 0.5120 - mse: 27.7449 - mae: 4.4294 - accuracy: 0.8093 - root_mean_squared_error: 5.2673 - val_loss: 0.4983 - val_mse: 27.3854 - val_mae: 4.3824 - val_accuracy: 0.8198 - val_root_mean_squared_error: 5.2331
Epoch 3/20
1500/1500 [==============================] - 22s 15ms/step - loss: 0.4682 - mse: 27.7471 - mae: 4.4294 - accuracy: 0.8242 - root_mean_squared_error: 5.2676 - val_loss: 0.5209 - val_mse: 27.3962 - val_mae: 4.3824 - val_accuracy: 0.8162 - val_root_mean_squared_error: 5.2341
Epoch 4/20
1500/1500 [==============================] - 21s 14ms/step - loss: 0.4419 - mse: 27.7485 - mae: 4.4294 - accuracy: 0.8376 - root_mean_squared_error:

...

2023-06-06 10:58:41.731825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 3s 8ms/step - loss: 0.4062 - mse: 27.6822 - mae: 4.4200 - accuracy: 0.8533 - root_mean_squared_error: 5.2614
Test Accuracy: [0.4062318205833435, 27.682172775268555, 4.419997215270996, 0.8532999753952026, 5.261385917663574]
313/313 [==============================] - 1s 4ms/step


/var/folders/mk/t6y50d7s2ds93t8l2lw18p_00000gn/T/ipykernel_15121/653596370.py:255: MatplotlibDeprecationWarning: Auto-removal of overlapping axes is deprecated since 3.6 and will be removed two minor releases later; explicitly call ax.remove() as needed.
  plt.subplot(5, 5, i + 1)


<Figure size 1500x1500 with 0 Axes>

<Figure size 1200x1400 with 0 Axes>

In [ ]:
model_name = "MobileNet"
number_of_epochs = 20

AL = AutomaticLearning(train_images, train_labels, test_images, test_labels, model_name + " to " + str(number_of_epochs) + " epochs", model_name)
AL.buildData()
AL.buildModel()
AL.train(number_of_epochs)
AL.plot(model_name, save = True, show = False)
result = AL.gather_data()

In [38]:
model_name = "DenseNet"
number_of_epochs = 20

AL = AutomaticLearning(train_images, train_labels, test_images, test_labels, model_name + " to " + str(number_of_epochs) + " epochs", model_name)
AL.buildData()
AL.buildModel()
AL.train(number_of_epochs)
AL.plot(model_name, save = True, show = False)
result = AL.gather_data()


Epoch 1/20


/Users/pablo/Library/Python/3.9/lib/python/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-06-06 10:28:54.803444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1500/1500 [==============================] - ETA: 0s - loss: 1.7891 - mse: 27.6891 - mae: 4.4294 - accuracy: 0.2411 - root_mean_squared_error: 5.2620

2023-06-06 10:30:08.515207: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1500/1500 [==============================] - 94s 58ms/step - loss: 1.7891 - mse: 27.6891 - mae: 4.4294 - accuracy: 0.2411 - root_mean_squared_error: 5.2620 - val_loss: 1.6859 - val_mse: 27.3366 - val_mae: 4.3824 - val_accuracy: 0.2573 - val_root_mean_squared_error: 5.2284
Epoch 2/20
1500/1500 [==============================] - 77s 52ms/step - loss: 1.6382 - mse: 27.6930 - mae: 4.4294 - accuracy: 0.2808 - root_mean_squared_error: 5.2624 - val_loss: 1.5747 - val_mse: 27.3370 - val_mae: 4.3824 - val_accuracy: 0.3268 - val_root_mean_squared_error: 5.2285
Epoch 3/20
1500/1500 [==============================] - 86s 57ms/step - loss: 1.6019 - mse: 27.6941 - mae: 4.4294 - accuracy: 0.3008 - root_mean_squared_error: 5.2625 - val_loss: 1.5506 - val_mse: 27.3385 - val_mae: 4.3824 - val_accuracy: 0.3210 - val_root_mean_squared_error: 5.2286
Epoch 4/20
1500/1500 [==============================] - 86s 57ms/step - loss: 1.5665 - mse: 27.6950 - mae: 4.4294 - accuracy: 0.3212 - root_mean_squared_error:

KeyboardInterrupt: 

In [ ]:
model_name = "ResNet"
number_of_epochs = 20

AL = AutomaticLearning(train_images, train_labels, test_images, test_labels, model_name + " to " + str(number_of_epochs) + " epochs", model_name)
AL.buildData()
AL.buildModel()
AL.train(number_of_epochs)
AL.plot(model_name, save = True, show = False)
result = AL.gather_data()

# Runs using SGD

In [ ]:
model_name = "CNN"
number_of_epochs = 20

AL = AutomaticLearning(train_images, train_labels, test_images, test_labels, model_name + " to " + str(number_of_epochs) + " epochs", model_name)
AL.buildData()
AL.buildModel()
AL.train(number_of_epochs)
AL.plot(model_name, save = True, show = False)
result = AL.gather_data()

# Tests

In [5]:
train_images = train_images / 255.0
test_images = test_images / 255.0


In [6]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])


In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

opt = optimizers.Adam(learning_rate=0.01)

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer = opt, metrics=['mse', 'mae','accuracy', tf.keras.metrics.RootMeanSquaredError()])


In [10]:
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels))


Epoch 1/10
   1/1875 [..............................] - ETA: 8:20 - loss: 67.1922 - mse: 42.0496 - mae: 5.9375 - accuracy: 0.8125 - root_mean_squared_error: 6.4846

2023-06-06 09:30:22.300548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


 739/1875 [==========>...................] - ETA: 11s - loss: 1.4198 - mse: 27.7255 - mae: 4.4232 - accuracy: 0.7212 - root_mean_squared_error: 5.2655

KeyboardInterrupt: 

In [61]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)


313/313 - 2s - loss: 0.3562 - accuracy: 0.8904 - 2s/epoch - 5ms/step

Test accuracy: 0.8903999924659729
